### 图的应用：拓扑排序 Topological Sort

- 从工作流程图得到工作次序排列的算法，称为“拓扑排序”
- 拓扑排序处理一个DAG，输出顶点的线性序列，使得两个顶点v,w，如果G中有(v,w)边，在线性序列中v就出现在w之前
- 拓扑排序广泛应用在**依赖事件的排期**上，还可以用在项目管理、数据库查询优化和矩阵乘法的次序优化上

拓扑排序可以采用DFS很好地实现：
- 将工作流程建立为图，工作项是节点，依赖关系是有向边
- 工作流程图一定是个DAG图，否则有循环依赖对DAG图调用DFS算法，以得到每个顶点的“结束时间”
- 按照每个顶点的“结束时间”从大到小排序输出这个次序下的顶点列表

### 图的应用：强连通分支 Strongly Connected Components

网页的互相超链接关联，可能会产生某种聚集，可以猜想，Web的底层结构可能存在某些同类网站的聚集

在图中发现**高度聚集节点群**的算法，即寻找“强连通分支”算法

强连通分支，定义为图G的一个子集C：C中的任意两个顶点v,w之间都有**路径**来回，即(v,w)(w,v)都是C的**路径**，而且C是具有这样性质的**最大**子集

先引入一个转置概念 Transposition

一个有向图G的转置$G^T$，定义为将图G的所有边的顶点交换次序，如将(v,w)转换为(w,v)，可以发现图和转置图在强连通分支的数量和划分上，是**相同**的

### 强连通分支算法：Kosaraju算法

1. 先对图G调用DFS算法，为每个顶点计算“结束时间”
2. 将图G进行转置，得到$G^T$
3. 再对$G^T$调用DFS算法，但在dfs函数中，对每个顶点的搜索循环里，要以顶点的“结束时间”的**倒序**顺序来搜索
4. 最后，深度优先森林中的每一个树就是一个强连通分支

### 图的应用：最短路径问题

假设将互联网路由器体系表示为一个带权边的图
- 路由器作为顶点，路由器之间网络连接作为边权重，可以包括网络连接的速度、网络负载程度、分时段优先级等影响因素
- 作为一个抽象，把所有影响因素合成为一个单一的权重
- 选择网络中传播速度最快的问题，就转变为在带全图上寻找最短路径的问题
- 这个问题与广度优先搜索BFS算法解决词梯问题相似，只是在边上增加了权重

### 最短路径算法

以发明者命名的Dijkstra算法
- 是一个迭代算法，得出从一个顶点到其余所有顶点的最短路径，很接近于广度优先搜索算法BFS的结果
- 具体实现上，在顶点Vertex累中的成员dist用于记录从开始顶点到本顶点的最短带权路径长度（权重之和），算法对途中的每个顶点迭代一次
    - 顶点的访问次序由一个**优先队列**来控制，队列中作为优先级的是顶点的dist属性
    - **开始顶点**的dist设为0，而其他所有顶点dist设为sys.maxsize(最大整数)，全部加入优先队列
    - 随着队列中每个最低dist顶点率先出队
    - 计算它与邻接顶点的权重，会引起其它顶点dist的减小和修改，引起堆重排
    - 并据更新后的dist优先级再依次出队

In [ ]:
import sys
sys.path.append('/home/maxq/projects/Python/ds/hashing_and_tree')
sys.path.append('/home/maxq/projects/Python/ds/graph')
from binary_heap import BinHeap
from graph import Graph, Vertex

def dijkstra(aGraph, start):
    pq = BinHeap()
    start.setDistance(0)
    pq.buildHeap([(v.getDistance(), v) for v in aGraph])
    while not pq.isEmpty():
        currentVert = pq.delMin()
        for nextVert in currentVert.getConnections():
            newDist = currentVert.getDistance() + currentVert.getWeight(nextVert)
            if newDist < nextVert.getDistance():
                nextVert.setDistance(newDist)
                nextVert.setPred(currentVert)
                pq.decreaseKey(nextVert, newDist)

### Dijkstra算法分析
- 将所有顶点加入优先队列并建堆，时间复杂度O(|V|)
- 每个顶点仅出队1次，每次delMin花费O(log|V|)，一共是O(|V|log|V|)
- 每个边关联到的顶点会做一次decreaseKey操作（O(log|V|)，一共是O（|E|log|V|)

以上相加得到时间复杂度O((|V|+|E|)log|V|)

### 应用：最小生成树

应用场景：

信息广播。网游或网络收音机，网游需要让所有玩家获知其它玩家所在的位置，收音机需要让所有听众获取直播的音频数据

信息广播的解决法1：单播解法

广播源维护一个收听者列表，将每条信息向收听者发送一次，每次信息都会采用最短路径算法到达收听者

缺点：在网络中的某些节点会多次重复处理相同信息，产生很多额外流量

信息广播的解决法2：洪水解法
广播源只发送一次，让所有的路由器都收到信息后，再转发给相邻的路由器和收听者

缺点：如果没有任何限制，这个方法将造成网络洪水灾难，很多路由器和收听者会不断重复收到相同的消息

针对缺点改进：给每条信息都附加一个生命值（TTL：Time To Live），初始设置为从消息源到最远的收听者的距离，每个路由器收到这条消息后，如果TTL大于0，则将TTL减少1，再转发出去，如果TTL等于0了，则直接抛弃这个消息。TTL的设置防止了灾难发生，但比单播产生的流量还要大

信息广播的最优解决法：最小生成树
路由器关系图上选取具有最小权重的生成树（minimum weight spanning tree）

生成树：拥有途中所有的顶点和最少数量的边，以保证连通的子图

图G(V, E)的最小生成树T，定义为包含所有顶点V，以及E的无圈子集，并且边权重之和最小

这样信息广播就可以从信息源开始，沿着最小生成树的路径层次向下传播，达到每个路由器只需要处理1次消息，同时总费用最小

### 最小生成树：Prim算法
- 属于“贪心算法”，每步都沿着最小权重边向前搜索
- 思路是如果T还不是生成树，则可以反复：找到一条最小权重的**可以安全添加**的边，将边添加到树T
- **可以安全添加**的边，定义为一端顶点在树中，另一端不在树中的边，以便保持树的无圈特性

In [ ]:
import sys
sys.path.append('/home/maxq/projects/Python/ds/hashing_and_tree')
sys.path.append('/home/maxq/projects/Python/ds/graph')
from binary_heap import BinHeap
from graph import Graph, Vertex

def prim(G, start):
    pq = BinHeap()
    for v in G:
        v.setDistance(sys.maxsize)
        v.setPred(None)
    start.setDistance(0)
    pq.buildHeap([(v.getDistance(), v) for v in G])
    while not pq.isEmpty():
        currentVert = pq.delMin()
        for nextVert in currentVert.getConnections():
            if newxVert in pq and newCost < nextVert.getDistance():
                nextVert.setPred(currentVert)
                nextVert.setDistance(newCost)
                pq.decreaseKey(nextVert, newCost)
